In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
plt.style.use('seaborn-colorblind')
%matplotlib inline

# Почему отсутствующие данные важны при обучении модели


* Некоторые алгоритмы не могут работать при наличии пропущенных значений.
* Даже для алгоритмов, которые учитывают отсутствующие данные, без их обработки модель может привести к неточным выводам.

Важно понимать механизмы, по которым отсутствующие поля появляются в наборе данных.

* Отсутствие полностью случайно
* Отсутствие случайно
* Отсутствие не случайно - зависит от ненаблюдаемых предикторов
* Отсутствие не случайно - зависит от самого пропущенного значения

### Как предположить механизм отсутствия

Можно определить механизм отсутствия по следующим критериям:

1. **Понимание бизнес-логики.** Во многих ситуациях можно предположить механизм, исследовав логику, лежащую в основе этой переменной.
2. **Статистический тест.** Разделите набор данных на те, в которых есть/нет пропусков, и выполните t-тест, чтобы определить, есть ли значимые различия. Если они есть, можно предположить, что отсутствие не является случайным.


## Загрузка данных

In [2]:
use_cols = [
    'Pclass', 'Sex', 'Age', 'Fare', 'SibSp',
    'Survived'
]

data = pd.read_csv('./data/titanic.csv', usecols=use_cols)
print(data.shape)
data.head(8)

(891, 6)


,Survived,Pclass,Sex,Age,SibSp,Fare
0,0,3,male,22.0,1,7.2500
1,1,1,female,38.0,1,71.2833
2,1,3,female,26.0,0,7.9250
3,1,1,female,35.0,1,53.1000
4,0,3,male,35.0,0,8.0500
5,0,3,male,NaN,0,8.4583
6,0,1,male,54.0,0,51.8625
7,0,3,male,2.0,3,21.0750


## Проверка пропущенных значений


Проверка количества и процента пропущенных значений для каждого признака

In [3]:
result = pd.concat([data.isnull().sum(),data.isnull().mean()],axis=1)
result.rename(index=str,columns={0:'total missing',1:'proportion'})

,total missing,proportion
Survived,0,0.000000
Pclass,0,0.000000
Sex,0,0.000000
Age,177,0.198653
SibSp,0,0.000000
Fare,0,0.000000


## Простое удаление

Удаление примеров, где пропущены значения

In [4]:
data2 = data.copy(deep=True)
data2 = data2.dropna(axis=0,inplace=False)
data2.shape

(714, 6)

## Дополнительный признак для обозначения пропущенного значения

In [5]:
# Создаем копию исходного датасета для безопасной обработки данных.
data3 = data.copy(deep=True)

# Задаем список столбцов, в которых мы хотим проверить наличие пропущенных значений.
NA_col = ['Age']

# Проходим по каждому столбцу из списка.
for i in NA_col:
    # Проверяем, есть ли пропущенные значения в текущем столбце.
    if data3[i].isnull().sum() > 0:
        # Если пропущенные значения есть, то создаем новый бинарный столбец, 
        # в котором 1 обозначает наличие пропуска, а 0 - отсутствие.
        data3[i + '_is_NA'] = np.where(data3[i].isnull(), 1, 0)
    else:
        # Если пропущенных значений в текущем столбце нет, выводим сообщение о его отсутствии.
        print("Столбец %s не содержит пропущенных значений" % i)

# Выводим первые 8 строк обработанного датасета для проверки.
data3.head(8)

,Survived,Pclass,Sex,Age,SibSp,Fare,Age_is_NA
0,0,3,male,22.0,1,7.2500,0
1,1,1,female,38.0,1,71.2833,0
2,1,3,female,26.0,0,7.9250,0
3,1,1,female,35.0,1,53.1000,0
4,0,3,male,35.0,0,8.0500,0
5,0,3,male,NaN,0,8.4583,1
6,0,1,male,54.0,0,51.8625,0
7,0,3,male,2.0,3,21.0750,0


## Заполнение пропущенного значения случайным числом

In [6]:
data4 = data.copy(deep=True)
NA_col = ['Age']
impute_value = -999
for i in NA_col:
    if data4[i].isnull().sum()>0:
        data4[i+'_'+str(impute_value)] = data4[i].fillna(impute_value)
    else:
        print("Column %s has no missing cases" % i)

In [7]:
data4.head(8)

,Survived,Pclass,Sex,Age,SibSp,Fare,Age_-999
0,0,3,male,22.0,1,7.2500,22.0
1,1,1,female,38.0,1,71.2833,38.0
2,1,3,female,26.0,0,7.9250,26.0
3,1,1,female,35.0,1,53.1000,35.0
4,0,3,male,35.0,0,8.0500,35.0
5,0,3,male,NaN,0,8.4583,-999.0
6,0,1,male,54.0,0,51.8625,54.0
7,0,3,male,2.0,3,21.0750,2.0


## Заполнение пропущенного значения медианой/модой/средним

In [8]:
print(data["Age"].mean())
print(data["Age"].median())
print(data["Age"].mode())

29.69911764705882
28.0
0    24.0
dtype: float64


In [9]:
data5 = data.copy(deep=True)
NA_col = ['Age']
strategy = 'median'
for i in NA_col:
    if data5[i].isnull().sum()>0:
        if strategy=='mean':
            data5[i+'_impute_mean'] = data5[i].fillna(data[i].mean())
        elif strategy=='median':
            data5[i+'_impute_median'] = data5[i].fillna(data[i].median())
        elif strategy=='mode':
            data5[i+'_impute_mode'] = data5[i].fillna(data[i].mode()[0])
    else:
        print("Column %s has no missing" % i)
        
data5.head(5)

,Survived,Pclass,Sex,Age,SibSp,Fare,Age_impute_median
0,0,3,male,22.0,1,7.2500,22.0
1,1,1,female,38.0,1,71.2833,38.0
2,1,3,female,26.0,0,7.9250,26.0
3,1,1,female,35.0,1,53.1000,35.0
4,0,3,male,35.0,0,8.0500,35.0


##  Заполнением хвостами расправления

Заполнение NA, значениями которые находятся в конце распределения - $\mu + 3*\sigma$



In [10]:
data6 = data.copy(deep=True)
NA_col = ['Age']
for i in NA_col:
    if data6[i].isnull().sum()>0:
        data6[i+'_impute_end_of_distr'] = data6[i].fillna(data[i].mean()+3*data[i].std())
    else:
        print("Column %s has no missing" % i)
        
data6.head(8)

,Survived,Pclass,Sex,Age,SibSp,Fare,Age_impute_end_of_distr
0,0,3,male,22.0,1,7.2500,22.00000
1,1,1,female,38.0,1,71.2833,38.00000
2,1,3,female,26.0,0,7.9250,26.00000
3,1,1,female,35.0,1,53.1000,35.00000
4,0,3,male,35.0,0,8.0500,35.00000
5,0,3,male,NaN,0,8.4583,73.27861
6,0,1,male,54.0,0,51.8625,54.00000
7,0,3,male,2.0,3,21.0750,2.00000


##  Случайное заполнение

Заполнение случайным значением из доступного множества трейн значений

In [11]:
# Создаем копию исходного датасета для безопасной обработки данных.
data7 = data.copy(deep=True)

# Задаем список столбцов, в которых мы хотим заполнить пропущенные значения случайными данными.
NA_col = ['Age']

# Проходим по каждому столбцу из списка.
for i in NA_col:
    # Проверяем, есть ли пропущенные значения в текущем столбце.
    if data7[i].isnull().sum() > 0:
        # Создаем новый столбец, который будет содержать заполненные случайными данными пропуски из оригинального столбца.
        data7[i + '_random'] = data7[i]
        
        # Извлекаем случайную выборку (random_sample) из непустых значений текущего столбца для заполнения пропусков.
        random_sample = data7[i].dropna().sample(data7[i].isnull().sum(), random_state=42)
        
        # Устанавливаем индексы пропущенных значений (NaN) равными индексам в random_sample.
        random_sample.index = data7[data7[i].isnull()].index
        
        # Заполняем пропущенные значения в новом столбце значениями из random_sample.
        data7.loc[data7[i].isnull(), str(i) + '_random'] = random_sample
    else:
        # Если пропущенных значений в текущем столбце нет, выводим сообщение об этом.
        print("Столбец %s не содержит пропущенных значений" % i)

# Выводим первые 8 строк обработанного датасета для проверки.
data7.head(8)


,Survived,Pclass,Sex,Age,SibSp,Fare,Age_random
0,0,3,male,22.0,1,7.2500,22.0
1,1,1,female,38.0,1,71.2833,38.0
2,1,3,female,26.0,0,7.9250,26.0
3,1,1,female,35.0,1,53.1000,35.0
4,0,3,male,35.0,0,8.0500,35.0
5,0,3,male,NaN,0,8.4583,42.0
6,0,1,male,54.0,0,51.8625,54.0
7,0,3,male,2.0,3,21.0750,2.0
